# Dynamic Programming Patterns

### Overview

Dynamic programming is used to solve problems where the solution to the overall problem relies on the solution to overlapping subproblems.

A classic example are the Fibonacci numbers, defined as:

`fib(n) = fib(n-1) + fib(n-2)`.

Since we need to recursively solve for `1...n`, we can tell caching the subproblems will be advantageous.

For example:

```
fib(4) = fib(3) + fib(2)

fib(3) = fib(2) + fib(1)
fib(2) = fib(1) + fib(0)
```

As we can see, `fib(2)` is repeated, as is `fib(1)`.

If the optimal solution to a problem can be found by finding the optimal solution to its subproblems we call this the __optimal substructure property__.

### Dynamic Programming Approaches

#### 1. Top-down with Memoization

With this approach, we set up a cache to store every result. Before jumping into our calculation, we check if an answer exists in the memoization cache.

```python
def calculate_fibonacci(n: int):
  memoization_cache = [-1 for x in range(n+1)]
  return _calculate_fibonacci(memoization_cache, n)


def _calculate_fibonacci(memoization_cache: list[int], n: int) -> int:
  if n < 2:
    return n

    # if we have already solved this subproblem, simply return the result from the cache
  if memoization_cache[n] >= 0:
    return memoization_cache[n]

  memoization_cache[n] = _calculate_fibonacci(memoization_cache, n-1) +
    _calculate_fibonacci(memoization_cache, n-2)
  
    return memoization_cache[n]
```

#### 2. Bottom-up Tabulation

With this approach, we start from the simplest subproblem and then build our way up to the solution by filling in a table.

```python
def calculate_fibonacci(n):
  dp = [0, 1]
  for i in range(2, n + 1):
    dp.append(dp[i - 1] + dp[i - 2])

  return dp[n]
```

#### Conclusion

Whichever technique we choose, we should use the same general approach:

1. Solve the problem using recursion
2. Apply top-down memoization or bottom-up tabulation

Writing out the recursion tree with method calls can often help you spot the solution.

### 0/1 Knapsack

(See other notes.)

### Unbounded Knapsack

These problems involve something like a knapsack but you can take the item __infinite__ times.

__Examples__

* Given a list of profits, weights and capacity, find the maximum profit IF the number of items you can take are unbounded.
* Given a rod of length ‘n’, we are asked to cut the rod and sell the pieces in a way that will maximize the profit. We are also given the price of every piece of length.
* Given an infinite supply of ‘n’ coin denominations and a total money amount, we are asked to find the total number of distinct ways to make up that amount.
* Given an infinite supply of ‘n’ coin denominations and a total money amount, we are asked to find the minimum number of coins needed to make up that amount.
* We are given a ribbon of length ‘n’ and a set of possible ribbon lengths. We need to cut the ribbon into the maximum number of pieces that comply with the above-mentioned possible lengths. Write a method that will return the count of pieces.

#### General Approach

The general approach is to do a DFS with backtracking. In other words, recurse with the item, but don't increase the index, and then recurse without the item, but do increase the index.

Then you can compare these two things or sum them.

```python
def rod_cutting(lengths: list[int], prices: list[int], remaining_length: int) -> int:
    """
    Given a rod of length ‘n’, we are asked to cut the rod and sell the pieces in a way that
    will maximize the profit. We are also given the price of every piece of length ‘i’ where
    ‘1 <= i <= n’.

    Example:
    Lengths: [1, 2, 3, 4, 5]
    Prices: [2, 6, 7, 10, 13]
    Rod Length: 5

    5xlength 1 = 10 profit
    2xlength 2 + 1xlength 1 = 14 profit
    1xlength 3 + 2xlength 2 = 11 profit
    etc.
    """
    return _rod_cutting(lengths, prices, remaining_length, 0)


def _rod_cutting(lengths: list[int], prices: list[int], remaining_length: int, index: int) -> int:
    """Recursive helper function."""
    if index >= len(lengths) or remaining_length <= 0:
        return 0

    profit_with_this_item = 0

    # Try taking this item
    if lengths[index] <= remaining_length:
        profit_with_this_item = prices[index] + _rod_cutting(lengths, prices, remaining_length-lengths[index], index)

    profit_without_this_item = _rod_cutting(lengths, prices, remaining_length, index+1)

    return max(profit_with_this_item, profit_without_this_item)
```

```python
def minimum_coin_change(denominations: list[int], amount: int) -> int:
    """
    Given an infinite supply of ‘n’ coin denominations and a total money amount,
    we are asked to find the minimum number of coins needed to make up that amount.

    Example:
        Denominations: {1, 2, 3}, Amount: 5 -> 2 (2, 3)
    """
    return _minimum_coin_change(denominations, amount, 0)


def _minimum_coin_change(denominations: list[int], amount: int, index: int) -> int:
    """Recursive helper function."""
    if amount == 0:
        return 0

    if index >= len(denominations):
        return math.inf

    num_with_this_coin = math.inf

    if denominations[index] <= amount:
        temp = _minimum_coin_change(denominations, amount-denominations[index], index)
        if temp != math.inf:  # Don't include if we overshot
            num_with_this_coin = temp + 1

    num_without_this_coin = _minimum_coin_change(denominations, amount, index+1)

    return min(num_with_this_coin, num_without_this_coin)
```

#### Optimizing Topdown

Here, we just create a cache of index & target and then check if we have that already when taking the item

```python
def rod_cutting_topdown(lengths: list[int], prices: list[int], remaining_length: int) -> int:
    """
    Given a rod of length ‘n’, we are asked to cut the rod and sell the pieces in a way that
    will maximize the profit. We are also given the price of every piece of length ‘i’ where
    ‘1 <= i <= n’.

    Example:
    Lengths: [1, 2, 3, 4, 5]
    Prices: [2, 6, 7, 10, 13]
    Rod Length: 5

    5xlength 1 = 10 profit
    2xlength 2 + 1xlength 1 = 14 profit
    1xlength 3 + 2xlength 2 = 11 profit
    etc.
    """
    # Top-down memoization, takes O(n*remaining_length) time and space
    dp = [[-1 for _ in range(remaining_length+1)] for _ in range(len(lengths))]
    return _rod_cutting_topdown(lengths, prices, remaining_length, 0, dp)


def _rod_cutting_topdown(
        lengths: list[int], prices: list[int], remaining_length: int, index: int, dp: list[list[int]]) -> int:
    """Recursive helper function."""
    if index >= len(lengths) or remaining_length <= 0:
        return 0

    profit_with_this_item = 0

    # Try taking this item
    if dp[index][remaining_length] == -1:
        if lengths[index] <= remaining_length:
            profit_with_this_item = prices[index] + _rod_cutting_topdown(lengths, prices, remaining_length-lengths[index], index, dp)

        profit_without_this_item = _rod_cutting_topdown(lengths, prices, remaining_length, index+1, dp)

        dp[index][remaining_length] = max(profit_with_this_item, profit_without_this_item)

    return dp[index][remaining_length]
```

```python
def minimum_coin_change_topdown(denominations: list[int], amount: int) -> int:
    """
    Given an infinite supply of ‘n’ coin denominations and a total money amount,
    we are asked to find the minimum number of coins needed to make up that amount.

    Example:
        Denominations: {1, 2, 3}, Amount: 5 -> 2 (2, 3)
    """
    dp = [[-1 for _ in range(amount+1)] for _ in range(len(denominations))]
    return _minimum_coin_change_topdown(denominations, amount, 0, dp)


def _minimum_coin_change_topdown(denominations: list[int], amount: int, index: int, dp: list[list[int]]) -> int:
    """Recursive helper functions."""
    if amount == 0:
        return 1

    if index >= len(denominations):
        return math.inf

    if dp[index][amount] == -1:
        num_with_this_coin = math.inf

        if denominations[index] <= amount:
            temp = _minimum_coin_change(denominations, amount - denominations[index], index)
            if temp != math.inf:  # Don't include if we overshot
                num_with_this_coin = temp + 1

        num_without_this_coin = _minimum_coin_change(denominations, amount, index + 1)

        dp[index][amount] = min(num_with_this_coin, num_without_this_coin)

    return dp[index][amount]
```

#### Optimizing Bottomup

In this case we create a matrix of item index (rows) x target (columns). We check if we should take the row above, or take the value from this row by subtracting target-current value.

```python
def rod_cutting_bottomup(lengths: list[int], prices: list[int], remaining_length: int) -> int:
    """
    Given a rod of length ‘n’, we are asked to cut the rod and sell the pieces in a way that
    will maximize the profit. We are also given the price of every piece of length ‘i’ where
    ‘1 <= i <= n’.

    Example:
    Lengths: [1, 2, 3, 4, 5]
    Prices: [2, 6, 7, 10, 13]
    Rod Length: 5

    5xlength 1 = 10 profit
    2xlength 2 + 1xlength 1 = 14 profit
    1xlength 3 + 2xlength 2 = 11 profit
    etc.
    """
    # Bottom-up memoization, takes O(n*remaining_length) time and space
    rows = len(lengths)

    dp = [[-1 for _ in range(remaining_length+1)] for _ in range(rows)]

    for row in range(rows):
        dp[row][0] = 0  # For 0 length, we have 0 profit

    for item in range(rows):
        for length in range(1, remaining_length+1):
            price_with_this_item = 0
            price_without_this_item = 0
            if lengths[item] <= length:
                price_with_this_item = prices[item] + dp[item][length - lengths[item]]
            if item > 0:
                price_without_this_item = dp[item-1][length]

            dp[item][length] = max(price_with_this_item, price_without_this_item)

    return dp[rows-1][remaining_length]
```

```python
def minimum_coin_change_bottomup(denominations: list[int], amount: int) -> int:
    """
    Given an infinite supply of ‘n’ coin denominations and a total money amount,
    we are asked to find the minimum number of coins needed to make up that amount.

    Example:
        Denominations: {1, 2, 3}, Amount: 5 -> 2 (2, 3)
    """
    dp = [[math.inf for _ in range(amount+1)] for _ in range(len(denominations))]

    for i in range(len(denominations)):
        dp[i][0] = 0

    for index in range(len(denominations)):
        for amt in range(amount+1):
            amount_with_this_item = math.inf

            if index > 0:
                # Get the amount excluding this item.
                dp[index][amt] = dp[index-1][amt]
            if denominations[index] <= amt:
                if dp[index][amt-denominations[index]] != math.inf:
                    # Get the amount with this coin (+1)
                    dp[index][amt] = min(dp[index-1][amt], dp[index][amt-denominations[index]]+1)

    return -1 if dp[len(denominations)-1][amount] == math.inf else dp[len(denominations)-1][amount]

```

### Fibonacci Numbers

This pattern involves evaluating the previous numbers in the sequence in some way. For example, with Fibonacci numbers, we sum them, but we may want to find the min or max instead.

__Examples__
* Write a function to calculate the nth Fibonacci number
* Given a stair with ‘n’ steps, implement a method to count how many possible ways are there to reach the top of the staircase, given that, at every step you can either take 1 step, 2 steps, or 3 steps.
* Given a number ‘n’, implement a method to count how many possible　ways there are to express ‘n’ as the sum of 1, 3, or 4.
* Given an array of positive numbers, where each element represents the max number of jumps that can be made forward from that element, write a program to find the minimum number of jumps needed to reach the end of the array (starting from the first element).
* Given a staircase with ‘n’ steps and an array of ‘n’ numbers representing the fee that you have to pay if you take the step. Implement a method to calculate the minimum fee required to reach the top of the staircase (beyond the top-most step). At every step, you have an option to take either 1 step, 2 steps, or 3 steps. You should assume that you are standing at the first step.
* There are n houses built in a line. A thief wants to steal the maximum possible money from these houses. The only restriction the thief has is that he can’t steal from two consecutive houses, as that would alert the security system. How should the thief maximize his stealing?

#### General Approach

The general approach is to recurse with the different options and choose the max/min/sum, or whatever the condition is.

Try drawing out a call tree with the sum to make sense of this.

```python
def staircase(n: int) -> int:
    """
    Given a stair with ‘n’ steps, implement a method to count how
    many possible ways are there to reach the top of the staircase,
    given that, at every step you can either take 1 step, 2 steps,
    or 3 steps.

    Example:
        Stairs: 3 -> 4 (1, 1, 1), (1, 2), (2, 1), (3)

    Time: O(3^n)
    """
    if n == 0:
        return 1  # Reached the top, so 1 way to get there

    if n == 1:
        return 1  # Only 1 step to the end, so return 1

    if n == 2:
        return 2  # We can take two 1-steps or one 2-step

    num_ways = 0
    num_ways += staircase(n-1)
    num_ways += staircase(n-2)
    num_ways += staircase(n-3)

    return num_ways


def staircase_topdown(n: int) -> int:
    """
    Given a stair with ‘n’ steps, implement a method to count how
    many possible ways are there to reach the top of the staircase,
    given that, at every step you can either take 1 step, 2 steps,
    or 3 steps.

    Example:
        Stairs: 3 -> 4 (1, 1, 1), (1, 2), (2, 1), (3)

    Time: O(n)
    Space: O(n)
    """
    dp = [-1 for _ in range(n+1)]
    return _staircase_topdown(n, dp)


def _staircase_topdown(n: int, dp: list[int]) -> int:
    """Recursive helper function."""
    if n < 0:
        return 0

    if n == 0:
        return 1

    if n == 1:
        return 1

    if n == 2:
        return 2

    if dp[n] != -1:
        return dp[n]

    dp[n] = _staircase_topdown(n-1, dp) + _staircase_topdown(n-2, dp) + _staircase_topdown(n-3, dp)

    return dp[n]


def staircase_bottomup(n: int) -> int:
    """
    Given a stair with ‘n’ steps, implement a method to count how
    many possible ways are there to reach the top of the staircase,
    given that, at every step you can either take 1 step, 2 steps,
    or 3 steps.

    Example:
        Stairs: 3 -> 4 (1, 1, 1), (1, 2), (2, 1), (3)

    The intuition for this one is that if we look at the topdown pattern,
    we can see that we sum up the previous 3 values.

    Time: O(n)
    Space: O(n)
    [0, 1, 2, 3, 4]
     1  1  2  4
    """
    dp = [1, 1, 2]
    if n < 3:
        return dp[n]

    for i in range(3, n+1):
        dp.append(dp[i-1] + dp[i-2] + dp[i-3])

    return dp[n]
```

```python
def minimum_jumps_with_fee(stairs: list[int]) -> int:
    """
    Given a staircase with ‘n’ steps and an array of ‘n’ numbers representing the fee that
    you have to pay if you take the step. Implement a method to calculate the minimum fee
    required to reach the top of the staircase (beyond the top-most step). At every step,
    you have an option to take either 1 step, 2 steps, or 3 steps. You should assume that
    you are standing at the first step.

    Example:
        [1, 2, 5, 2, 1, 1] -> 3 (0 > 3 > end) Cost=1+2==3
    """
    return _minimum_jumps_with_fee(stairs, 0)


def _minimum_jumps_with_fee(stairs: list[int], index: int) -> int:
    """Recursive helper function."""

    # In this problem, the aim is to beyond the top of the staircase
    if index >= len(stairs):
        return 0

    # If we had to reach the top element, we could check if we overran and return
    # math.inf, for example, and then check the result before taking min.

    minimum_fee = math.inf

    for i in range(1, 4):
        result = _minimum_jumps_with_fee(stairs, index+i)
        minimum_fee = min(stairs[index] + result, minimum_fee)

    return minimum_fee


def minimum_jumps_with_fee_topdown(stairs: list[int]) -> int:
    """
    Given a staircase with ‘n’ steps and an array of ‘n’ numbers representing the fee that
    you have to pay if you take the step. Implement a method to calculate the minimum fee
    required to reach the top of the staircase (beyond the top-most step). At every step,
    you have an option to take either 1 step, 2 steps, or 3 steps. You should assume that
    you are standing at the first step.

    Example:
        [1, 2, 5, 2, 1, 1] -> 3 (0 > 3 > end) Cost=1+2==3
    """
    dp = [-1 for _ in range(len(stairs))]
    return _minimum_jumps_with_fee_topdown(stairs, 0, dp)


def _minimum_jumps_with_fee_topdown(stairs: list[int], index: int, dp: list[int]) -> int:
    """Recursive helper function."""
    if index >= len(stairs):
        return 0

    if dp[index] != -1:
        return dp[index]

    minimum_jumps = math.inf

    for i in range(1, 4):
        result = _minimum_jumps_with_fee_topdown(stairs, index+i, dp)
        minimum_jumps = min(result+stairs[index], minimum_jumps)

    dp[index] = minimum_jumps

    return dp[index]


def minimum_jumps_with_fee_bottomup(stairs: list[int]) -> int:
    """
    Given a staircase with ‘n’ steps and an array of ‘n’ numbers representing the fee that
    you have to pay if you take the step. Implement a method to calculate the minimum fee
    required to reach the top of the staircase (beyond the top-most step). At every step,
    you have an option to take either 1 step, 2 steps, or 3 steps. You should assume that
    you are standing at the first step.

    Example:
        [1, 2, 5, 2, 1, 1] -> 3 (0 > 3 > end) Cost=1+2==3
    """
    # dp=[0, 1, 1, 0, 0, 0], 1 and 2 can be reached for cost of first square
    # i=2, dp=[0, 1, 1, 0, 0, 0], cost to reach 3 is the min of:
    #       (cost to reach 2 + cost of 2, cost to reach 1 + cost of 1, cost to reach 0 + cost of 0)
    # i=2, dp=[0, 1, 1, 1, 0, 0, 0]
    # i=3, dp=[0, 1, 1, 1, 3, 0, 0]
    # i=4, dp=[0, 1, 1, 1, 3, 3, 0]
    # etc.
    # We take the minimum of the previous 3 costs/indices.
    dp = [0 for _ in range(len(stairs)+1)]

    dp[0] = 0  # No fee for 0 steps
    dp[1] = stairs[0]  # Only 1 step: we have to pay it's fee
    dp[2] = stairs[0]  # We HAVE to pay the fee for the first step, and we can reach
    # second step from there.

    for i in range(2, len(stairs)):
        dp[i+1] = min(stairs[i] + dp[i],
                      stairs[i-1] + dp[i-1],
                      stairs[i-2] + dp[i-2])

    return dp[len(stairs)]
```

```python
def house_thief(houses: list[int]) -> int:
    """
    There are n houses built in a line. A thief wants to steal the maximum possible
    money from these houses. The only restriction the thief has is that he can’t
    steal from two consecutive houses, as that would alert the security system.
    How should the thief maximize his stealing?

    Example:
        [2, 5, 1, 3, 6, 2, 4] -> 15 (steal from houses worth 5, 6, 4)
    """
    # Solution: Try to recurse from this index, and next index and see which is worth more
    # It's the same as prev try with this, and then try without it.
    return _house_thief(houses, 0)


def _house_thief(houses: list[int], index: int) -> int:
    """Recursive helper function."""
    if index >= len(houses):
        return 0

    with_this_house = houses[index] + _house_thief(houses, index+2)
    without_this_house = _house_thief(houses, index+1)

    return max(with_this_house, without_this_house)


def house_thief_topdown(houses: list[int]) -> int:
    """
    There are n houses built in a line. A thief wants to steal the maximum possible
    money from these houses. The only restriction the thief has is that he can’t
    steal from two consecutive houses, as that would alert the security system.
    How should the thief maximize his stealing?

    Example:
        [2, 5, 1, 3, 6, 2, 4] -> 15 (steal from houses worth 5, 6, 4)
    """
    # Solution: Try to recurse from this index, and next index and see which is worth more
    # It's the same as prev try with this, and then try without it.
    # As we can see, we recurse from the same indices several times
    dp = [-1 for _ in range(len(houses))]
    return _house_thief_topdown(houses, 0, dp)


def _house_thief_topdown(houses: list[int], index: int, dp: list[list[int]]) -> int:
    """Recursive helper function."""
    if index >= len(houses):
        return 0

    if dp[index] != -1:
        return dp[index]

    with_this_house = houses[index] + _house_thief_topdown(houses, index+2, dp)
    without_this_house = _house_thief_topdown(houses, index+1, dp)

    dp[index] = max(with_this_house, without_this_house)

    return dp[index]


def house_thief_bottomup(houses: list[int]) -> int:
    """
    There are n houses built in a line. A thief wants to steal the maximum possible
    money from these houses. The only restriction the thief has is that he can’t
    steal from two consecutive houses, as that would alert the security system.
    How should the thief maximize his stealing?

    Example:
        [2, 5, 1, 3, 6, 2, 4] -> 15 (steal from houses worth 5, 6, 4)

    houses=[2, 5, 1, 3]
    dp=[0, 2, 0, 0, 0]
    > For each element, take max(previous house,
    """
    dp = [0 for _ in range(len(houses)+1)]
    dp[0] = 0  # 0 houses, so no value
    dp[1] = houses[0]  # If only 1 house, take that

    for i in range(2, len(houses)+1):
        # At each stage, we either take the previous-2 element + cost, or the previous element.
        dp[i] = max(dp[i-2] + houses[i-1], dp[i-1])

    return dp[len(houses)]
```

--


# Break


### Palindromic Subsequence

This pattern involved finding palindromic subsequences or substrings. A subsequence is a sequence that can be derived from another sequence by deleting some or no elements without changing the order of the remaining elements.

__Examples:__

* Given a sequence, find the length of its Longest Palindromic Subsequence (LPS). In a palindromic subsequence, elements read the same backward and forward. A subsequence is a sequence that can be derived from another sequence by deleting some or no elements without changing the order of the remaining elements.
* Given a string, find the length of its Longest Palindromic Substring (LPS). In a palindromic string, elements read the same backward and forward.
* Given a string, find the total number of palindromic substrings in it. Please note we need to find the total number of substrings and not subsequences.
* Given a string, find the minimum number of characters that we can remove to make it a palindrome.
* Given a string, we want to cut it into pieces such that each piece is a palindrome. Write a function to return the minimum number of cuts needed.


#### General Approach

There are a few general approaches.

If we want to find subsequence:
* If the current start and end and palindromic, add 2 + recurse, moving start and end
* If not the same, try recursing skipping start and skipping end and take the max

If we want to find substring:
* Similar, but when both are the same we need to check if the remaining chars are also a palindrome. We do this by recursing and checking if the result matches the length of remaining chars.

If we want to find __count__ of palindromic substrings:
* Subtract the palindromes between start and end, and also use a helper function to check if palindromic.

```python
def minimum_deletions_topdown(s: str) -> int:
    """
    Given a string, find the minimum number of characters that we can remove to make
    it a palindrome.

    abdbca -> 1 (remove c to give abdba)
    cddpd -> 2 (remove c, d to give ddd)
    pqr -> 2 (remove any two to give a single char, p, q, or r)
    
    # Similar (Same approach can solve):
        "Find longest palindromic subsequence" (just don't do len(s)-"
        "Minimum insertions to make a string palindroic"
        "Is a string palindromic if we make K insertions?"
    """
    # Observation: The num of chars would be total chars - longest palindromic seq
    dp = [[-1 for _ in range(len(s))] for _ in range(len(s))]
    return len(s) - _minimum_deletions_topdown(s, 0, len(s)-1, dp)


def _minimum_deletions_topdown(s: str, start: int, end: int, dp: list[list[int]]) -> int:
    """Recursive helper function."""
    if start > end:
        return 0

    if start == end:
        return 1   # len(1) char is palindromic

    if dp[start][end] == -1:

        if s[start] == s[end]:
            dp[start][end] = 2 + _minimum_deletions_topdown(s, start+1, end-1, dp)
        else:
            skip_start = _minimum_deletions_topdown(s, start+1, end, dp)
            skip_end = _minimum_deletions_topdown(s, start, end-1, dp)

            dp[start][end] = max(skip_start, skip_end)

    return dp[start][end]

```

```python
def longest_palindromic_substr_topdown(s: str) -> int:
    """
    Given a string, find the length of its Longest Palindromic Substring (LPS). In
    a palindromic string, elements read the same backward and forward.

    Example: cddpd -> 3 (dpd)
    """
    # Well, here is the recursive solution:
    # If the two chars match, then check the remaining chars
    # If we find the result matches the len(remaining chars), whole thing can be returned
    # Else return the max of skipping the first or second.
    dp = [[-1 for _ in range(len(s))] for _ in range(len(s))]
    return _longest_palindromic_substr_topdown(s, 0, len(s)-1, dp)


def _longest_palindromic_substr_topdown(s: str, start: int, end: int, dp: list[list[int]]) -> int:
    """Recursive helper function."""
    if start > end:
        return 0

    if start == end:
        return 1

    if dp[start][end] == -1:
        if s[start] == s[end]:
            remaining_chars = end-start-1
            result = _longest_palindromic_substr_topdown(s, start+1, end-1, dp)
            if result == remaining_chars:
                dp[start][end] = 2 + result
                return dp[start][end]

        skip_start = _longest_palindromic_substr_topdown(s, start+1, end, dp)
        skip_end = _longest_palindromic_substr_topdown(s, start, end-1, dp)

        dp[start][end] = max(skip_start, skip_end)

    return dp[start][end]
```

```python
def _count_palindromic_substr(s: str, start: int, end: int) -> int:
    """Recursive helper function."""
    if start > end:
        return 0

    if start == end:
        return 1  # Every len(1) str is a palindrome.

    count = 0
    if _is_palindromic(s, start, end):
        count += 1

    count += _count_palindromic_substr(s, start+1, end)
    count += _count_palindromic_substr(s, start, end-1)
    count -= _count_palindromic_substr(s, start+1, end-1)  # Subtract double-counted palindromes

    return count


def _is_palindromic(s: str, start: int, end: int) -> bool:
    """Recursive helper function."""
    while start <= end:
        if s[start] != s[end]:
            return False

        start += 1
        end -= 1

    return True
```

### Longest Common Substring

This pattern involves finding commonality between 2 strings. For example, the longest common substring, subsequence, edit distance, and so on.

__Examples__

* Given two strings ‘s1’ and ‘s2’, find the length of the longest substring which is common in both the strings.
*  Given two strings ‘s1’ and ‘s2’, find the length of the longest subsequence which is common in both the strings. A subsequence is a sequence that can be derived from another sequence by deleting some or no elements without changing the order of the remaining elements. (s1=abdca, s2=cbda -> 3 (bda))
* Given strings s1 and s2, we need to transform s1 into s2 by deleting and inserting characters. Write a function to calculate the count of the minimum number of deletion and insertion operations.
*  Given a number sequence, find the length of its Longest Increasing Subsequence (LIS). In an increasing subsequence, all the elements are in increasing order (from lowest to highest).
* Given a number sequence, find the increasing subsequence with the highest sum. Write a method that returns the highest sum.
* Given a numberic sequence, find the minimum number of elements that should be deleted to make the remaining sequence sorted.
* Given a string and a pattern, write a method to count the number of times the pattern appears in the string as a subsequence.
* Calculate the edit distance between 2 strings
* Given three strings ‘m’, ‘n’, and ‘p’, write a method to find out if ‘p’ has been formed by interleaving ‘m’ and ‘n’. ‘p’ would be considered interleaving ‘m’ and ‘n’ if it contains all the letters from ‘m’ and ‘n’ and the order of letters is preserved too.

#### General Approach

Generally, what we want to do is hold an index for each of the strings, and if the match recurse with 1 + next index. Or try recursing with each index alternatively. Also: if we want to find something like minimum insertions / deletions, consider that this will be related the longest common subsequence. If we need to do something like find maximum increasing sequence, we need to store the previous largest index. If it increases, we try recursing again with this index, and we try in any case skipping this index.

```python
def longest_common_substring(s1: str, s2: str) -> int:
    """
    Given two strings ‘s1’ and ‘s2’, find the length of the longest substring
    which is common in both the strings.

    Example:
        s1=abdca, s2=cbda -> 2 (bd)
        s1=passport, s2=ppsspt -> 3 (ssp)
    """
    # If the letters match, we return 1 + rest of string
    # Otherwise we try skipping 1 letter from each string.
    # Time: O(n^2)
    # Note: For subSEQUENCE, we just return when both match.
    dp = [[-1 for _ in range(len(s2))] for _ in range(len(s1))]
    return _longest_common_substring(s1, s2, 0, 0, dp)


def _longest_common_substring(s1: str, s2: str, s1_index: int, s2_index: int, dp: list[list[int]]) -> int:
    """Recursive helper function."""
    if s1_index >= len(s1) or s2_index >= len(s2):
        return 0

    if dp[s1_index][s2_index] == -1:

        skip_both = 0
        if s1[s1_index] == s2[s2_index]:
            skip_both = 1 + _longest_common_substring(s1, s2, s1_index+1, s2_index+2, dp)

        skip_s1 = _longest_common_substring(s1, s2, s1_index+1, s2_index, dp)
        skip_s2 = _longest_common_substring(s1, s2, s1_index, s2_index+1, dp)

        dp[s1_index][s2_index] = max(skip_both, skip_s1, skip_s2)

    return dp[s1_index][s2_index]
```

```python
def min_deletions_insertions(s1: str, s2: str) -> tuple[int, int]:
    """
    Given strings s1 and s2, we need to transform s1 into s2 by deleting and
    inserting characters. Write a function to calculate the count of the minimum
    number of deletion and insertion operations.

    Example:
        s1=abdca, s2=cbda -> 2,1 (delete a and c -- 2, and insert c -- 1)
    """
    # Solution: This is the same problem as longest common subsequence
    # The difference is we do longest string - common subseq and shorted string - common subseq
    dp = [[-1 for _ in range(len(s2))] for _ in range(len(s1))]
    longest_subseq = _min_deletions_insertions(s1, s2, 0, 0, dp)

    longest = s1 if len(s1) > len(s2) else s2
    shortest = s1 if len(s1) < len(s2) else s2
    # Deletions and insertions
    return len(longest)-longest_subseq, len(shortest)-longest_subseq


def _min_deletions_insertions(s1: str, s2: str, s1_index: int, s2_index: int, dp: list[list[int]]) -> int:
    """Recursive helper function. (longest common subsequence)"""
    if s1_index >= len(s1) or s2_index >= len(s2):
        return 0

    if dp[s1_index][s2_index] == -1:
        if s1[s1_index] == s2[s2_index]:
            dp[s1_index][s2_index] = 1 + _min_deletions_insertions(s1, s2, s1_index+1, s2_index+1, dp)
        else:
            skip_s1 = _min_deletions_insertions(s1, s2, s1_index+1, s2_index, dp)
            skip_s2 = _min_deletions_insertions(s1, s2, s1_index, s2_index+1, dp)

            dp[s1_index][s2_index] = max(skip_s1, skip_s2)

    return dp[s1_index][s2_index]
```

```python
def longest_increasing_subsequence_topdown(nums: list[int]) -> int:
    """
    Given a number sequence, find the length of its Longest Increasing Subsequence
    (LIS). In an increasing subsequence, all the elements are in increasing order
    (from lowest to highest).

    Example:
        [4, 2, 3, 6, 10, 1, 12] -> 5 (2, 3, 6, 10, 12)
        [-4, 10, 3, 7, 15] -> 4 (-4, 3, 7, 15)
        
    Similar: Find the number of deletions to sort.
    """
    dp = [[-1 for _ in range(len(nums))] for _ in range(len(nums))]
    return _longest_increasing_subsequence_topdown(nums, 0, -1, dp)


def _longest_increasing_subsequence_topdown(
        nums: list[int], current_index: int, previous_index: int, dp: list[list[int]]) -> int:
    if current_index >= len(nums):
        return 0  # We at least have 1 number in our sequence

    if dp[current_index][previous_index] == -1:

        with_this_index = 0
        if previous_index == -1 or nums[current_index] > nums[previous_index]:
            with_this_index = 1 + _longest_increasing_subsequence(nums, current_index+1, current_index)

        skip_this_index = _longest_increasing_subsequence(nums, current_index+1, previous_index)

        dp[current_index][previous_index] = max(with_this_index, skip_this_index)

    return dp[current_index][previous_index]
```